In [21]:
import xgboost
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [22]:
data = pd.read_csv('preprocessing/data/london_clean.csv')
data = data.loc[
    :,
    [
        "DateOfCall",
        "PropertyType",
        "NumPumpsAttending",
        "PumpHoursRoundUp",
        "CostCat",
        "mean_temp",
    ],
]
X = data.drop(["CostCat"], axis=1).values  # Input features (attributes)
y = data["CostCat"].values  # Target vector


X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, test_size=0.3, random_state=42
)


In [23]:
final_model = xgboost.XGBClassifier(objective="binary:logistic", random_state=42)
final_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = final_model.predict(X_test)

# Evaluate the accuracy of the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.90      0.82    210199
           1       0.76      0.48      0.59    127674
           2       1.00      1.00      1.00     31404
           3       0.70      0.91      0.79      4963
           4       0.72      0.42      0.53      4546
           5       0.91      0.96      0.93      7200

    accuracy                           0.77    385986
   macro avg       0.80      0.78      0.78    385986
weighted avg       0.77      0.77      0.75    385986

